In [14]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
import keras
from keras.layers import Dense
from keras.models import Sequential
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from sklearn.metrics import roc_auc_score


In [8]:
train = pd.read_csv('Task2_Train_Data_pro.csv', low_memory = False, delimiter =',')
pre_target = train['Pred_Target'].values
train.drop(['Buy_sesID'],axis=1,inplace=True )
train.drop(['Unnamed: 0'],axis=1,inplace=True )
train.head()

,Cli_ItemID,ttl_Cli,AVG_CLI,Item_CLI,Fre_Cli_Itm,IN_Fre_Ctg,Cli_Wd,Cli_T,Dur_Sec,Cli_buy,Item_bought_P90,Fir_Cli,Las_Cli,Item_avg_pric,AVG_Pric,Pri_ctg_P90,Pred_Target,below_ctg_avg_pric,below_ctg_P90
0,214851326.0,8.0,2.0,3.0,1.0,1.0,5.0,3.0,255.0,3.0,1.0,0.0,0.0,1046.0,1923.025449,4920.6,1.0,1.0,1.0
1,214853248.0,8.0,2.0,1.0,0.0,0.0,5.0,3.0,0.0,1.0,1.0,0.0,0.0,837.0,2061.555292,48380.5,0.0,1.0,1.0
2,214853094.0,8.0,2.0,1.0,0.0,1.0,5.0,3.0,0.0,1.0,1.0,0.0,0.0,4188.0,1923.025449,4920.6,0.0,0.0,1.0
3,214853090.0,8.0,2.0,3.0,1.0,1.0,5.0,3.0,3117.0,3.0,1.0,1.0,1.0,4188.0,1923.025449,4920.6,1.0,0.0,1.0
4,214567333.0,4.0,1.0,1.0,0.0,1.0,7.0,2.0,0.0,1.0,1.0,0.0,0.0,837.0,2061.555292,48380.5,1.0,1.0,1.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355403 entries, 0 to 355402
Data columns (total 19 columns):
Cli_ItemID            355403 non-null float64
ttl_Cli               355403 non-null float64
AVG_CLI               355403 non-null float64
Item_CLI              355403 non-null float64
Fre_Cli_Itm           355403 non-null float64
IN_Fre_Ctg            355403 non-null float64
Cli_Wd                355403 non-null float64
Cli_T                 355403 non-null float64
Dur_Sec               355403 non-null float64
Cli_buy               355403 non-null float64
Item_bought_P90       355403 non-null float64
Fir_Cli               355403 non-null float64
Las_Cli               355403 non-null float64
Item_avg_pric         355403 non-null float64
AVG_Pric              355403 non-null float64
Pri_ctg_P90           355403 non-null float64
Pred_Target           355403 non-null float64
below_ctg_avg_pric    355403 non-null float64
below_ctg_P90         355403 non-null float64
dtypes: float64

In [9]:
target = to_categorical(pre_target)
train.drop(['Pred_Target'], axis = 1, inplace = True)

In [10]:
predictors = train.values
n_cols = predictors.shape[1]
#print(n_cols)

In [11]:
model = Sequential()
model.add(Dense (32,activation='relu', input_shape = (n_cols,)))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(predictors,target)

predictions =model.predict(train)

predicted_prob_true = predictions[:,1]

print(predicted_prob_true)

Epoch 1/1
355403/355403 [==============================] - 13s 37us/step - loss: 7.4016 - acc: 0.5408
[0. 0. 0. ... 0. 0. 0.]


In [91]:
def get_new_model(n_cols):
    model = Sequential()
    model.add(Dense (32,activation='relu', input_shape = (n_cols,)))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(predictors,target)
    return(model)

In [92]:
# Import the SGD optimizer
from keras.optimizers import SGD

# Create list of learning rates: lr_to_test
lr_to_test = [0.000001,0.01,1]

# Loop over learning rates
for lr in lr_to_test:
    print('\n\nTesting model with learning rate: %f\n'%lr )
    
    # Build new model to test, unaffected by previous models
    model = get_new_model(n_cols)
    
    # Create SGD optimizer with specified learning rate: my_optimizer
    my_optimizer = SGD(lr=lr)
    
    # Compile the model
    model.compile(optimizer=my_optimizer,loss='categorical_crossentropy')
    
    # Fit the model
    model.fit(predictors,target)




Testing model with learning rate: 0.000001

Epoch 1/1
355403/355403 [==============================] - 12s 33us/step - loss: 7.4016 - acc: 0.5408
Epoch 1/1
355403/355403 [==============================] - 11s 32us/step - loss: 7.4016


Testing model with learning rate: 0.010000

Epoch 1/1
355403/355403 [==============================] - 14s 38us/step - loss: 8.7165 - acc: 0.4592
Epoch 1/1
355403/355403 [==============================] - 14s 40us/step - loss: 8.7165


Testing model with learning rate: 1.000000

Epoch 1/1
355403/355403 [==============================] - 17s 47us/step - loss: 7.4016 - acc: 0.5408
Epoch 1/1
355403/355403 [==============================] - 13s 36us/step - loss: 7.4016


In [15]:
roc_auc_score(pre_target,predicted_prob_true)

0.5

Validation

In [29]:
Valid = pd.read_csv('Task2_Validation_Data_pro.csv', low_memory = False, delimiter =',')
pre_target_V = Valid['Pred_Target'].values
Valid.drop(['Buy_sesID'], axis =1, inplace = True)
Valid.drop(['Unnamed: 0'], axis =1, inplace = True)
Valid.drop(['Pred_Target'], axis =1, inplace = True)
Valid.head()

,Cli_ItemID,ttl_Cli,AVG_CLI,Item_CLI,Fre_Cli_Itm,IN_Fre_Ctg,Cli_Wd,Cli_T,Dur_Sec,Cli_buy,Item_bought_P90,Fir_Cli,Las_Cli,Item_avg_pric,AVG_Pric,Pri_ctg_P90,below_ctg_avg_pric,below_ctg_P90
0,214854358.0,5.0,2.5,2.0,0.0,1.0,4.0,3.0,149.0,3.0,1.0,1.0,1.0,1360.0,1923.025449,4920.6,1.0,1.0
1,214859126.0,5.0,2.5,3.0,1.0,1.0,4.0,3.0,946.0,3.0,1.0,1.0,1.0,1570.0,1923.025449,4920.6,1.0,1.0
2,214712244.0,3.0,1.0,1.0,0.0,1.0,3.0,4.0,0.0,1.0,1.0,1.0,1.0,947.0,2061.555292,48380.5,1.0,1.0
3,214676480.0,3.0,1.0,1.0,0.0,1.0,3.0,4.0,0.0,1.0,1.0,1.0,1.0,5538.0,2061.555292,48380.5,0.0,1.0
4,214690839.0,3.0,1.0,1.0,0.0,1.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,686.0,2061.555292,48380.5,1.0,1.0


In [34]:
V_predictions =model.predict(Valid)
y_pre_V = V_predictions[:,1]
print(pre_target_V)
print(y_pre_V)
roc_auc_score(pre_target_V,y_pre_V)

[0. 1. 1. ... 1. 1. 1.]
[0. 0. 0. ... 0. 0. 0.]


0.5